In [1]:
def load_train_data(path):
    tokens_list = []
    labels_list = []

    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            label_str, text = line.split("\t")
            tokens = text.split(" ")
            labels = label_str.split(" ")
            assert len(tokens) == len(labels)
            tokens_list.append(tokens)
            labels_list.append(labels)

    return tokens_list, labels_list

train_tokens, train_labels = load_train_data("train/train.tsv")


In [2]:
def load_test_data(in_path, label_path):
    with open(in_path, encoding="utf-8") as f_in, open(label_path, encoding="utf-8") as f_lab:
        tokens_list = [line.strip().split() for line in f_in]
        labels_list = [line.strip().split() for line in f_lab]
        return tokens_list, labels_list

test_tokens, test_labels = load_test_data("dev-0/in.tsv", "dev-0/expected.tsv")


In [3]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
y_pred = []

for tokens in test_tokens:
    preds = ner_pipeline(tokens)
    
    predicted_labels = ["O"] * len(tokens)
    flat_preds = [ent for group in preds for ent in group]  # spłaszczenie listy

    for entity in flat_preds:
        idx = entity.get("index", None)
        if idx is None:
            continue
        predicted_labels[idx - 1] = entity["entity_group"]  # np. "PER", "ORG"
    
    # Zamień entity_group na BIO-style (uproszczenie, do dalszego dopracowania)
    bio_labels = []
    prev_tag = "O"
    for i, tag in enumerate(predicted_labels):
        if tag == "O":
            bio_labels.append("O")
        elif i == 0 or predicted_labels[i - 1] != tag:
            bio_labels.append("B-" + tag)
        else:
            bio_labels.append("I-" + tag)
    y_pred.append(bio_labels)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KeyboardInterrupt: 

In [ ]:
from seqeval.metrics import classification_report

print(classification_report(test_labels, y_pred))
